In [2]:
cd ..

/home/pierre/git/semisupervised


In [5]:
import numpy as np
from probnum.quad import bayesquad
from probnum.quad.integration_measures import IntegrationMeasure, GaussianMeasure

from probnum.quad.solvers import BayesianQuadrature, BQIterInfo
from probnum.quad.typing import DomainLike
from probnum.quad._bayesquad import _check_domain_measure_compatibility

from probnum.randprocs.kernels import Kernel
from probnum.randvars import Normal
from typing import Callable, Optional, Tuple



In [6]:
def bayesquad_from_initial_data(
    fun: Callable,
    nodes: np.ndarray,
    fun_evals: np.ndarray,
    kernel: Optional[Kernel] = None,
    measure: Optional[IntegrationMeasure] = None,
    domain: Optional[DomainLike] = None,
    policy: Optional[str] = "bmc",
    rng: Optional[np.random.Generator] = None,
    options: Optional[dict] = None
) -> Tuple[Normal, BQIterInfo]:
    r"""Perform Bayesian Quadraturefrom a given set of nodes and function
    evaluations.
    Parameters
    ----------
    nodes
        *shape=(n_eval, input_dim)* -- Locations at which the function evaluations are
        available as ``fun_evals``.
    fun_evals
        *shape=(n_eval,)* -- Function evaluations at ``nodes``.
    kernel
        The kernel used for the GP model. Defaults to the ``ExpQuad`` kernel.
    measure
        The integration measure. Defaults to the Lebesgue measure.
    domain
        The integration domain. Contains lower and upper bound as scalar or
        ``np.ndarray``. Obsolete if ``measure`` is given.
    options
        A dictionary with the following optional solver settings
            scale_estimation : Optional[str]
                Estimation method to use to compute the scale parameter. Defaults
                to 'mle'. Options are
                ==============================  =======
                 Maximum likelihood estimation  ``mle``
                ==============================  =======
            jitter : Optional[FloatLike]
                Non-negative jitter to numerically stabilise kernel matrix
                inversion. Defaults to 1e-8.
    Returns
    -------
    integral :
        The integral belief subject to the provided measure or domain.
    info :
        Information on the performance of the method.
    Raises
    ------
    ValueError
        If neither a domain nor a measure are given.
    Warns
    -----
    UserWarning
        When ``domain`` is given but not used.
    See Also
    --------
    bayesquad : Computes the integral using an acquisition policy.
    Warnings
    --------
    Currently the method does not support tuning of the kernel parameters
    other than the global kernel scale. Hence, the method may perform poorly unless the
    kernel parameters are set to appropriate values by the user.
    """

    if nodes.ndim != 2:
        raise ValueError(
            "The nodes must be given a in an array with shape=(n_eval, input_dim)"
        )

    input_dim, domain, measure = _check_domain_measure_compatibility(
        input_dim=nodes.shape[1], domain=domain, measure=measure
    )

    bq_method = BayesianQuadrature.from_problem(
        input_dim=input_dim,
        kernel=kernel,
        measure=measure,
        domain=domain,
        policy=policy,
        initial_design=None,
        options=options,
    )

    # Integrate
    integral_belief, bqstate, info = bq_method.integrate(
        fun=fun, nodes=nodes, fun_evals=fun_evals, rng=rng
    )

    return integral_belief, bqstate, info

In [7]:

input_dim = 1
domain = (0, 1)
def fun(x):
    return x.reshape(-1, )
F, info = bayesquad(fun, input_dim, domain=domain, rng=np.random.default_rng(0), policy="ivr_rand")
print(F.mean)


0.500024114110369


In [13]:
nodes = np.linspace(0, 1, 5)[:, None]
fun_evals = nodes.reshape(-1, )
fun = lambda x: x.reshape(-1, )
kernel = None
measure = GaussianMeasure(0., 1.)
domain = None  # Obsolete if measure is given
policy = "ivr_rand" #"mi" "ivr" "us" "us_rand", "mi_rand", "ivr_rand"
options = {"max_evals": 11, "batch_size": 5} # 

In [14]:
F, bqstate, info = bayesquad_from_initial_data(fun=fun,nodes=nodes, fun_evals=fun_evals, measure=measure, policy=policy, options=options, rng=np.random.default_rng(0))

In [15]:
bqstate.nodes

array([[ 0.        ],
       [ 0.25      ],
       [ 0.5       ],
       [ 0.75      ],
       [ 1.        ],
       [-1.40152021],
       [-2.39823287],
       [ 2.55242403],
       [-1.47913783],
       [ 1.17245123],
       [-0.68949746]])

In [ ]:
if nodes.ndim != 2:
    raise ValueError(
        "The nodes must be given a in an array with shape=(n_eval, input_dim)"
    )

input_dim, domain, measure = _check_domain_measure_compatibility(
    input_dim=nodes.shape[1], domain=domain, measure=measure
)

bq_method = BayesianQuadrature.from_problem(
    input_dim=input_dim,
    kernel=kernel,
    measure=measure,
    domain=domain,
    policy=None,
    initial_design=None,
    options=options,
)

# Integrate
integral_belief, bq_state, info = bq_method.integrate(
    fun=None, nodes=nodes, fun_evals=fun_evals, rng=None
)